In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [ ]:
label = pd.read_csv('../data/training_set_labels.csv')
label.head()

In [ ]:
train = pd.read_csv('../data/training_set_features.csv')
train['h1n1_target'] = label['h1n1_vaccine']
train['seasonal_target'] = label['seasonal_vaccine']
train.head()

In [ ]:
test = pd.read_csv('../data/test_set_features.csv')
test.head()

In [ ]:
submission_df = pd.DataFrame(test['respondent_id'])
submission_df.head()

## Data preprocessing

In [ ]:
train.drop(columns=['respondent_id'], inplace=True)
test.drop(columns=['respondent_id'], inplace=True)

In [ ]:
# One hot encoding in categorical columns in 0 and 1
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# Transformar True y False en 1 y 0
train = train * 1
test = test * 1

In [ ]:
# Comprobar las características más importantes según un Random Forest
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador de Random Forest
rf_h1n1 = RandomForestClassifier(n_estimators=1000, random_state=42)

# Entrenar el modelo
rf_h1n1.fit(train.drop(['h1n1_target', 'seasonal_target'], axis=1), train['seasonal_target'])

# Obtener la importancia de las características
importances = rf_h1n1.feature_importances_

# Crear un DataFrame con las características y su importancia
feature_importances = pd.DataFrame({'feature': train.drop(['h1n1_target', 'seasonal_target'], axis=1).columns, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False).reset_index(drop=True)

# Mostrar las características que sobrepasen un umbral
threshold = 0.015
selected_features = feature_importances[feature_importances['importance'] > threshold]

In [ ]:
# Seleccionar las características en el conjunto de entrenamiento y de prueba
train_h1n1 = train[list(selected_features['feature'])+ ['h1n1_target']]
test_h1n1 = test[list(selected_features['feature'])]

In [ ]:
# Comprobar las características más importantes según un Random Forest
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador de Random Forest
rf_seasonal = RandomForestClassifier(n_estimators=1000, random_state=42)

# Entrenar el modelo
rf_seasonal.fit(train.drop(['h1n1_target', 'seasonal_target'], axis=1), train['seasonal_target'])

# Obtener la importancia de las características
importances = rf_seasonal.feature_importances_

# Crear un DataFrame con las características y su importancia
feature_importances = pd.DataFrame({'feature': train.drop(['h1n1_target', 'seasonal_target'], axis=1).columns, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False).reset_index(drop=True)

# Mostrar las características que sobrepasen un umbral
threshold = 0.015
selected_features = feature_importances[feature_importances['importance'] > threshold]

In [ ]:
# Seleccionar las características en el conjunto de entrenamiento y de prueba
train_seasonal = train[list(selected_features['feature'])+ ['seasonal_target']]
test_seasonal = test[list(selected_features['feature'])]

In [ ]:
# Imputar valores nulos con la media para las columnas numéricas
train_h1n1 = train_h1n1.fillna(train_h1n1.mean())
train_seasonal = train_seasonal.fillna(train_seasonal.mean())
test_h1n1 = test_h1n1.fillna(test_h1n1.mean())
test_seasonal = test_seasonal.fillna(test_seasonal.mean())

In [ ]:
X_train_h1n1 = train_h1n1.drop(['h1n1_target'], axis=1)
X_train_seasonal = train_seasonal.drop(['seasonal_target'], axis=1)
y_train_h1n1 = train_h1n1[['h1n1_target']]
y_train_seasonal = train_seasonal[['seasonal_target']]
X_test_h1n1 = test_h1n1
X_test_seasonal = test_seasonal

In [ ]:
print("El tamaño de X_train_h1n1 es: ", X_train_h1n1.shape)   
print("El tamaño de y_train_h1n1 es: ", y_train_h1n1.shape)
print("El tamaño de X_test_h1n1 es: ", X_test_h1n1.shape)
print("El tamaño de X_train_seasonal es: ", X_train_seasonal.shape)
print("El tamaño de y_train_seasonal es: ", y_train_seasonal.shape)
print("El tamaño de X_test_seasonal es: ", X_test_seasonal.shape)

## Neural Network

In [ ]:
import tensorflow as tf

### H1N1 Vaccine

In [ ]:
model_h1n1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[X_train_h1n1.shape[1]]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model_h1n1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_h1n1.summary()

In [ ]:
X_train_h1n1 = np.array(X_train_h1n1)
y_train_h1n1 = np.array(y_train_h1n1['h1n1_target'])

In [ ]:
X_train_h1n1.shape

In [ ]:
y_train_h1n1.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir X_train y y_train_seasonal en datos de entrenamiento y validación
X_train_h1n1_split, X_val_h1n1_split, y_train_h1n1_split, y_val_h1n1_split = train_test_split(X_train_h1n1, y_train_h1n1, test_size=0.2, random_state=42)

model_h1n1.fit(X_train_h1n1_split, y_train_h1n1_split,
          validation_data=(X_val_h1n1_split, y_val_h1n1_split),
          epochs=15)

In [ ]:
# Predict
predictions = model_h1n1.predict(X_test_h1n1)

In [ ]:
predictions.shape

In [ ]:
h1n1_df = pd.DataFrame(predictions, columns=['not_h1n1_vaccine', 'h1n1_vaccine'])
submission_df['h1n1_vaccine'] = h1n1_df['h1n1_vaccine']
submission_df.head()

### Seasonal Vaccine

In [ ]:
model_seasonal = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[X_train_seasonal.shape[1]]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model_seasonal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_seasonal.summary()

In [ ]:
X_train_seasonal = np.array(X_train_seasonal)
y_train_seasonal = np.array(y_train_seasonal['seasonal_target'])

In [ ]:
X_train_seasonal.shape

In [ ]:
y_train_seasonal.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir X_train y y_train_seasonal en datos de entrenamiento y validación
X_train_seasonal_split, X_val_seasonal_split, y_train_seasonal_split, y_val_seasonal_split = train_test_split(X_train_seasonal, y_train_seasonal, test_size=0.2, random_state=42)

model_seasonal.fit(X_train_seasonal_split, y_train_seasonal_split,
          validation_data=(X_val_seasonal_split, y_val_seasonal_split),
          epochs=15)

In [ ]:
# Predict
predictions = model_seasonal.predict(X_test_seasonal)

In [ ]:
seasonal_df = pd.DataFrame(predictions, columns=['not_seasonal_vaccine', 'seasonal_vaccine'])
submission_df['seasonal_vaccine'] = seasonal_df['seasonal_vaccine']
submission_df.head()

### Save in csv

In [ ]:
submission_df.to_csv('submission.csv', index=False)